In [ ]:
import os
os.chdir('..')

In [ ]:
pwd

In [ ]:
from scripts import labeling_rules

In [ ]:
import spacy
nlp = spacy.load('en_core_web_trf')

In [ ]:
nlp.add_pipe('labeling_ruler')
nlp.add_pipe('labeling_extend_ruler')
nlp.add_pipe('expand_entities')

In [ ]:
doc = nlp('On 3 April 2020 , in response to the covid-19 crisis , the government passed Law No . 21,218 , which created a monthly subsidy to guarantee a minimum monthly income of 300,000 pesos ( US$ 378 ) for employees who receive less than 384,363 pesos ( US$ 484 ) per month . ')
[(ent, ent.label_) for ent in doc.ents]

In [ ]:
nlp(']')[0].orth_

In [ ]:
list(doc.ents)

In [ ]:
end_index = 22
print(doc[end_index])
next((nc.end for nc in doc.noun_chunks if nc.start == end_index), None)

In [ ]:
[(tok, tok.ent_type_) for tok in nlp('Food systems contribute 19%–29% of global anthropogenic greenhouse gas (GHG) emissions, releasing 9,800–16,900 megatonnes of carbon dioxide equivalent (MtCO2e) in 2008.')]

In [ ]:
import srsly
for item in srsly.read_jsonl('corpus/sentences_dev.jsonl'):
    if item['label'] == 'INTERESTING':
        doc = nlp(item['text'])
        print(doc)
        print([(tok, tok.ent_type_) for tok in doc])
        print("====")

In [ ]:
from spacy.pipeline import EntityRuler
from scripts.labeling_rules import *
ruler = EntityRuler(nlp)
ruler.add_patterns([
    {
        "label": "IX_REFERENCE_VALUE",
        "pattern": [
            {"ent_type": "IX_REFERENCE_VALUE", "OP": "+"},
            {"ent_type": "", "OP": "*"},
            {"LOWER": {"IN": UNIT_LOWERS + TERMS_LOWERS}},
            {"LOWER": {"IN": ["]", ")"]}, "OP": "?"},
        ],
    },
])

In [ ]:
doc = nlp('More than 4 million ghg tco2e die prematurely from illness attributable to the household air pollution from cooking with solid fuels.')
spacy.displacy.serve(doc)

In [ ]:
[(tok, tok.ent_type_) for tok in doc]

In [ ]:
from spacy.matcher import DependencyMatcher
ruler = DependencyMatcher(nlp.vocab)
patterns = [
[
    # anchor token: founded
    {
        "RIGHT_ID": "noun",
        "RIGHT_ATTRS": {"POS": "NOUN"}
    },
    {
        "LEFT_ID": "noun",
        "RIGHT_ID": "num",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": "nummod"}
    },
    {
        "LEFT_ID": "num",
        "RIGHT_ID": "num2",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": {"IN":["compound", "quantmod"]}}, 
        "OP": "*"
    },
    {
        "LEFT_ID": "num2",
        "RIGHT_ID": "num3",
        "REL_OP": ">",
        "RIGHT_ATTRS": {"DEP": {"IN":["compound", "quantmod"]}}, 
        "OP": "*"
    }
],]
ruler.add("IX_REFERENCE_VALUE", patterns)

In [ ]:
ruler(doc)

In [ ]:
doc = nlp('Over 4 million people die prematurely from illness attributable to the household air pollution from cooking with solid fuels.')
for _, tok_inds in ruler(doc):
    print(doc[min(tok_inds) : max(tok_inds) + 1])